# Compare model training runs on dataset versions

## Introduction

You can version datasets, models, and other file objects as Artifacts in Neptune.

This guide shows how to:
* Keep track of the dataset version with Neptune artifacts
* See if models were trained on the same dataset version
* Compare datasets in the Neptune UI to see what changed

By the end of this guide, you will train a few models on different dataset versions and compare those versions in the Neptune UI.

[See this example in Neptune](https://app.neptune.ai/o/common/org/data-versioning/experiments?compare=IwdgNMQ&split=tbl&dash=artifacts&viewId=2b313653-1aa2-40e8-8bf2-cd13f0f96862&base=DAT-18&to=DAT-17)

![image](https://neptune.ai/wp-content/uploads/artifacts-compare-runs-on-dataset.png)

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client scikit-learn==0.24.1

## Prepare a model training script

As an example, we'll use a script that trains a scikit-learn model on the iris dataset.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

TRAIN_DATASET_PATH = "../datasets/tables/train.csv"
TEST_DATASET_PATH = "../datasets/tables/test.csv"

PARAMS = {
    "n_estimators": 5,
    "max_depth": 1,
    "max_features": 2,
}


def train_model(params, train_path, test_path):
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)

    FEATURE_COLUMNS = ["sepal.length", "sepal.width", "petal.length", "petal.width"]
    TARGET_COLUMN = ["variety"]
    X_train, y_train = train[FEATURE_COLUMNS], train[TARGET_COLUMN]
    X_test, y_test = test[FEATURE_COLUMNS], test[TARGET_COLUMN]

    rf = RandomForestClassifier(**params)
    rf.fit(X_train, y_train)

    return rf.score(X_test, y_test)

## Initialize Neptune and create new run

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project="common/data-versioning", api_token="ANONYMOUS")

Click on the link above to open this run in Neptune.

It's empty at the moment, but keep the tab with the run open to see what happens next.

**Few explanations**

In the above code You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new Run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `common/data-versioning` as an anonymous user `neptuner`, you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Get your [Neptune project name](https://docs.neptune.ai/getting-started/installation#setting-the-project-name)
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project='YOUR_WORKSPACE/YOUR_PROJECT', api_token='YOUR_API_TOKEN')
```

## Add tracking of the dataset version and parameters

Save datasets versions as Neptune artifacts

In [ ]:
run["datasets/train"].track_files(TRAIN_DATASET_PATH)
run["datasets/test"].track_files(TEST_DATASET_PATH)

**Note:**

You can also version the entire folder where your datasets by running

```python
run["datasets"].track_files(DATASET_FOLDER)
```

Also, people often keep track of datasets at the project level with [Project metadata](https://docs.neptune.ai/api-reference/project).

For more information see [Organize and share dataset versions](https://docs.neptune.ai/how-to-guides/data-versioning/organize-and-share-dataset-versions).

## Run model training and log parameters and metrics to Neptune

Now train a model and log the test score to Neptune

In [ ]:
run["parameters"] = PARAMS

score = train_model(PARAMS, TRAIN_DATASET_PATH, TEST_DATASET_PATH)

run["metrics/test_score"] = score

## Stop logging to the current run 
<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Change training dataset

Let's now change the training dataset that we'll be using

In [ ]:
TRAIN_DATASET_PATH = "../datasets/tables/train_v2.csv"

## Run model training on a new training dataset

Let's run model training again.
* Initialize the Neptune Run

In [ ]:
new_run = neptune.init(project="common/data-versioning", api_token="ANONYMOUS")

* Log dataset versions

In [ ]:
new_run["datasets/train"].track_files(TRAIN_DATASET_PATH)
new_run["datasets/test"].track_files(TEST_DATASET_PATH)

* Execute model training

In [ ]:
new_run["parameters"] = PARAMS

score = train_model(PARAMS, TRAIN_DATASET_PATH, TEST_DATASET_PATH)

new_run["metrics/test_score"] = score

Stop logging to currently active Neptune Run

In [ ]:
new_run.stop()

## Compare model training runs in the Neptune UI

To see that the score changed due to different dataset version: 
* Go to the `Runs table` in the Neptune UI
* Click on **+Add column**, type in 'artifacts/train' and click on it to add to the `Runs table`
* Click on the **Eye** icon and go to [Compare runs > Artifacts](https://docs.neptune.ai/you-should-know/comparing-runs#artifact) to see how the datasets changed

[See this example in Neptune](https://app.neptune.ai/o/common/org/data-versioning/experiments?compare=IwdgNMQ&split=tbl&dash=artifacts&viewId=2b313653-1aa2-40e8-8bf2-cd13f0f96862&base=DAT-18&to=DAT-17)

![image](https://neptune.ai/wp-content/uploads/artifacts-compare-runs-on-dataset.png)
